# Eat Safe, Love

## Notebook Set Up

In [19]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [20]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [21]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [22]:
# review the collections in our database
print(db.list_collection_names())


[]


In [23]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [28]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': {'$eq': 20}}

# Use count_documents to display the number of documents in the result
print(f'Establishments with hygiene score of 20: {establishments.count_documents(query)}')

# Display the first document in the results using pprint
first = establishments.find(query)
pprint(first[0])

Establishments with hygiene score of 20: 0


IndexError: no such item for Cursor instance

In [29]:
# Convert the result to a Pandas DataFrame
first_df = pd.DataFrame(first)
# Display the number of rows in the DataFrame
print(f'DF Rows: {len(first_df)}')
# Display the first 10 rows of the DataFrame
first_df.head(10)

DF Rows: 0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [30]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex': 'London'}, 'RatingValue': {'$gte': 4}}

# Use count_documents to display the number of documents in the result
print(f'Establishments in London with a rating >= 4: {establishments.count_documents(query)}')
# Display the first document in the results using pprint
london_rating = establishments.find(query)
pprint(london_rating[0])

Establishments in London with a rating >= 4: 0


IndexError: no such item for Cursor instance

In [ ]:
# Convert the result to a Pandas DataFrame
london_rating_df = pd.DataFrame(london_rating)
# Display the number of rows in the DataFrame
print(f'DF Rows: {len(london_rating_df)}')
# Display the first 10 rows of the DataFrame
london_rating_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = establishments.find_one({'BusinessName': 'Penang Flavours'})['geocode']['latitude']
longitude = establishments.find_one({'BusinessName': 'Penang Flavours'})['geocode']['longitude']

query = {
    'RatingValue': {'$eq': 5}, 
    'geocode.latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search},
    'geocode.longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search}
    }
sort = [('scores.Hygiene', 1)]
limit = 5

penang_proximity_results = list(establishments.find(query).sort(sort).limit(limit))
# Print the results

pprint(penang_proximity_results)


In [ ]:
# Convert result to Pandas DataFrame
penang_proximity_df = pd.DataFrame(penang_proximity_results)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_set = {'$match': {'scores.Hygiene': {'$eq': 0}}}
group_set = {'$group': {'_id': '$LocalAuthorityName',
                          'count': {'$sum': 1}
                          }}
sort_values = {'$sort': {'count': -1}}
pipeline = [match_set, group_set, sort_values]
final_query_results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f'Establishments with hygiene score = 0: {len(final_query_results)}')

# Print the first 10 results
pprint(final_query_results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
final_query_df = pd.DataFrame(final_query_results)

# Display the number of rows in the DataFrame
print(f'DF Rows: {len(final_query_df)}')

# Display the first 10 rows of the DataFrame
final_query_df.head(10)
